In [1]:
import os
import time
from datetime import datetime
import argparse
import gymnasium as gym
import numpy as np
import torch as th
import pandas as pd
import csv

from stable_baselines3 import PPO,SAC,TD3
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
from stable_baselines3.common.evaluation import evaluate_policy

from gym_pybullet_drones.utils.Logger import Logger
from gym_pybullet_drones.envs.HoverAviary import HoverAviary
from gym_pybullet_drones.envs.MultiHoverAviary import MultiHoverAviary
from gym_pybullet_drones.utils.utils import sync, str2bool
from gym_pybullet_drones.utils.enums import ObservationType, ActionType, Physics

from policies import GaussianMLPPolicy
#from server import Federated_RL

DEFAULT_GUI = True
DEFAULT_RECORD_VIDEO = True
DEFAULT_OUTPUT_FOLDER = 'results'
DEFAULT_COLAB = False
DEFAULT_DYNAMICS = Physics('pyb') # pyb: Pybullet dynamics; dyn: Explicit Dynamics specified in BaseAviary.py
DEFAULT_WIND = np.array([0, 0.05, 0]) # units are in induced newtons
DEFAULT_OBS = ObservationType('kin') # 'kin' or 'rgb'
DEFAULT_ACT = ActionType('one_d_rpm') # 'rpm' or 'pid' or 'vel' or 'one_d_rpm' or 'one_d_pid'
DEFAULT_AGENTS = 4
DEFAULT_MA = False
DEFAULT_MASS = 0.037 # Actual default is 0.027

DR = True
MASS_RANGE = [0.027, 0.042] # Maximum recommended payload is 15g
WIND_RANGE = 0.005 # Inspired by literature

# Maintain consistent network structures
policy_kwargs = dict(activation_fn=th.nn.Tanh,
                     net_arch=dict(pi=[512, 512, 256, 128], qf=[32, 32]))

pybullet build time: Jun 24 2024 15:23:59


In [2]:
def train(multiagent=DEFAULT_MA, output_folder=DEFAULT_OUTPUT_FOLDER, gui=DEFAULT_GUI, plot=True, colab=DEFAULT_COLAB, record_video=DEFAULT_RECORD_VIDEO, local=True):
    filename = 'TD3_test_run'
    if not os.path.exists(filename):
        os.makedirs(filename+'/')

    if not multiagent:
        train_env = make_vec_env(HoverAviary,
                                 env_kwargs=dict(obs=DEFAULT_OBS, act=DEFAULT_ACT),
                                 n_envs=1,
                                 seed=0
                                 )
        eval_env = HoverAviary(obs=DEFAULT_OBS, act=DEFAULT_ACT)
    else:
        train_env = make_vec_env(MultiHoverAviary,
                                 env_kwargs=dict(num_drones=DEFAULT_AGENTS, obs=DEFAULT_OBS, act=DEFAULT_ACT),
                                 n_envs=1,
                                 seed=0
                                 )
        eval_env = MultiHoverAviary(num_drones=DEFAULT_AGENTS, obs=DEFAULT_OBS, act=DEFAULT_ACT)

    #### Check the environment's spaces ########################
    print('[INFO] Action space:', train_env.action_space)
    print('[INFO] Observation space:', train_env.observation_space)

    #### Train the model #######################################
    model = PPO('MlpPolicy',
                train_env,
                # tensorboard_log=filename+'/tb/',
                policy_kwargs=policy_kwargs,
                local_iterations=2,
                verbose=1)
    
        #### Target cumulative rewards (problem-dependent) ##########
    if DEFAULT_ACT == ActionType.ONE_D_RPM:
        target_reward = 474.15 if not multiagent else 949.5
    else:
        target_reward = 467. if not multiagent else 920.
    callback_on_best = StopTrainingOnRewardThreshold(reward_threshold=target_reward,
                                                     verbose=1)
    eval_callback = EvalCallback(eval_env,
                                 callback_on_new_best=callback_on_best,
                                 verbose=1,
                                 best_model_save_path=filename+'/',
                                 log_path=filename+'/',
                                 eval_freq=int(1000),
                                 deterministic=True,
                                 DR=DR,
                                 mass_range=MASS_RANGE,
                                 wind_range=WIND_RANGE,
                                 render=False)
    
    model.learn(total_timesteps=int(1e7) if local else int(1e2), # shorter training in GitHub Actions pytest
                callback=eval_callback,
                log_interval=100)

    #### Save the model ########################################
    model.save(filename+'/final_model.zip')
    print(filename)
'''
    #### Print training progression ############################
    with np.load(filename+'/evaluations.npz') as data:
        for j in range(data['timesteps'].shape[0]):
            print(str(data['timesteps'][j])+","+str(data['results'][j][0]))
            '''

'\n    #### Print training progression ############################\n    with np.load(filename+\'/evaluations.npz\') as data:\n        for j in range(data[\'timesteps\'].shape[0]):\n            print(str(data[\'timesteps\'][j])+","+str(data[\'results\'][j][0]))\n            '

In [3]:
train()

/Users/kevinhan/opt/anaconda3/envs/drones/lib/python3.12/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


[INFO] Action space: Box(-1.0, 1.0, (1, 1), float32)
[INFO] Observation space: Box([[-inf -inf   0. -inf -inf -inf -inf -inf -inf -inf -inf -inf  -1.  -1.
   -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.]], [[inf inf inf inf inf inf inf inf inf inf inf inf  1.  1.  1.  1.  1.  1.
   1.  1.  1.  1.  1.  1.  1.  1.  1.]], (1, 27), float32)
Using cpu device


/Users/kevinhan/opt/anaconda3/envs/drones/lib/python3.12/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=1000, episode_reward=335.12 +/- 0.00
Episode length: 242.00 +/- 0.00
---------------------------------
| eval/              |          |
|    mean_ep_length  | 242      |
|    mean_reward     | 335      |
| time/              |          |
|    total_timesteps | 1000     |
---------------------------------
New best mean reward!
Eval num_timesteps=2000, episode_reward=335.12 +/- 0.00
Episode length: 242.00 +/- 0.00
---------------------------------
| eval/              |          |
|    mean_ep_length  | 242      |
|    mean_reward     | 335      |
| time/              |          |
|    total_timesteps | 2000     |
---------------------------------


KeyboardInterrupt: 

In [23]:
model = SAC.load('/Users/kevinhan/opt/anaconda3/envs/drones/lib/python3.12/site-packages/Federated_RL/SAC_test_run/final_model.zip')

In [11]:
for i in model.get_parameters()['policy']:
    print(i + ': ' + str(model.get_parameters()['policy'][i].shape))

log_std: torch.Size([1])
mlp_extractor.policy_net.0.weight: torch.Size([512, 27])
mlp_extractor.policy_net.0.bias: torch.Size([512])
mlp_extractor.policy_net.2.weight: torch.Size([512, 512])
mlp_extractor.policy_net.2.bias: torch.Size([512])
mlp_extractor.policy_net.4.weight: torch.Size([256, 512])
mlp_extractor.policy_net.4.bias: torch.Size([256])
mlp_extractor.policy_net.6.weight: torch.Size([128, 256])
mlp_extractor.policy_net.6.bias: torch.Size([128])
mlp_extractor.value_net.0.weight: torch.Size([32, 27])
mlp_extractor.value_net.0.bias: torch.Size([32])
mlp_extractor.value_net.2.weight: torch.Size([32, 32])
mlp_extractor.value_net.2.bias: torch.Size([32])
action_net.weight: torch.Size([1, 128])
action_net.bias: torch.Size([1])
value_net.weight: torch.Size([1, 32])
value_net.bias: torch.Size([1])


In [12]:
print(model.get_parameters()['policy']['action_net.bias'])

tensor([0.0088])
